# City Shadows: Google Street View Image Extraction
This notebook automates the collection of Google Street View images using the GSV API

### Requirements:
- .env file containing a **Google Street View** (https://developers.google.com/maps/documentation/streetview) API key (GSV_API)

## Import libraries

In [ ]:
import geopandas as gpd
import math
from dotenv import load_dotenv
import numpy as np
from scipy.interpolate import interp1d
import shapely
import urllib, os
import random
import requests
import time

## Reading shape files
shape - obtained from Geofabrik. contains all available road data in the country
small_makati - obtained from LiPAD (https://lipad.dream.upd.edu.ph/shapefilegen.html). geofabrik may have shapefiles of administrative areas for your respective country.

In [ ]:
shape=gpd.read_file("..\data\gis_osm_roads_free_1.shp")
makati = gpd.read_file("..\data\makati_city.shp")
small_makati = gpd.read_file("..\data\smaller_makati.shp")

OPTIONAL: Remove areas without any GSV info

In [ ]:
urdaneta = gpd.read_file("..\\data\\urdaneta.shp")
lower_makati = gpd.read_file("..\\data\\lower_makati.shp")

In [ ]:
only_gsv = shapely.difference(small_makati, urdaneta)
only_gsv = shapely.difference(only_gsv, lower_makati)
# for viewing
only_gsv.plot()

Extracts all the streets that are contained in the smaller administrative area.

In [ ]:
makati_streets = shape[makati['geometry'].item().contains(shape['geometry'])]
less_makati_streets = shape[small_makati['geometry'].item().contains(shape['geometry'])]

In [ ]:
less_makati_streets[0:850].plot()

Cost calculation<br>
Run this block if you wanna check how many points will be captured 

In [ ]:
sum = 0
FIFTY_METERS = 0.0000449 * 10
ANGLE_COUNT = 8
for street in less_makati_streets[250:850]['geometry'].items():
    sum += math.ceil(street[1].length/FIFTY_METERS)
required = sum * ANGLE_COUNT 
required


In [ ]:
PER_1000_IMAGE = 7
USD_TO_PHP = 55
required / 1000 * PER_1000_IMAGE 

## Variable Initialization

### Angles to record, API key, and output folder location

In [ ]:
angle = []
interval = 45
for i in range (0, 360, interval):
   angle.append(i)
load_dotenv(override=True)
GSV_API = os.getenv('GSV_API')
key = "&key=" + GSV_API  
myloc = "..\out"         

### Loading already processed pano_ids

In [ ]:
processed_pano_ids = []

if os.path.exists("pano_ids.txt"):
    with open("pano_ids.txt", "r") as f:
        processed_pano_ids = [line.strip() for line in f]

print(f"Loaded {len(processed_pano_ids)} pano_ids from pano_ids.txt.")

### Loading already processed coordinates

In [ ]:
processed_lat_long = []

if os.path.exists("lat_long.txt"):
    with open("lat_long.txt", "r") as f:
        processed_lat_long = [line.strip() for line in f]

print(f"Loaded {len(processed_lat_long)} latlong pairs from lat_long.txt.")

## Function Initialization

In [ ]:
def sleepRandom():
   time.sleep(random.randrange(5,7))


def getGSVMetadata(lat,long):
  sleepRandom()
  metaURL = "https://maps.googleapis.com/maps/api/streetview/metadata?location=" + str(lat) + "," + str(long) + key
  urllib.request.urlretrieve(metaURL)
  response = requests.get(metaURL)

  # check json response for status and pano_id
  if response.status_code == 200:
      data = response.json()
      pano_id = data.get("pano_id")
      status = data.get("status")

      if pano_id in processed_pano_ids:
        return "DUPLICATE"

      if status == "OK":
        return pano_id
      else:
        if(str(lat) + "," + str(long)) not in processed_lat_long:       # check if lat,long is already processed
          processed_lat_long.append(str(lat) + "," + str(long))         # append lat,long to processed latlong list
          with open("lat_long.txt", "a") as f:                          # append lat,long to .txt file
              f.write(f"{str(lat) + "," + str(long)}\n")
        return "NO IMAGE"

  else:
      print(f"Error: {response.status_code}, {response.text}")

def getStreet(add,outputLocation, angles):
  base = "https://maps.googleapis.com/maps/api/streetview?size=640x640&fov=120&source=outdoor&pitch=20&location="
  coor = add

  # loop through angles declared

  for angle in angles:
    sleepRandom()
    heading = "&heading=" + str(angle)    
    finalURL = base + coor + heading + key
    fi = add + "_" + str(angle) + ".jpg"
    urllib.request.urlretrieve(finalURL, os.path.join(outputLocation,fi))
  
def getEquidistantPoints(pathLength, x, y):
  # Linear length on the line; euclidean distance
  distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
  distance = distance/distance[-1]

  fx, fy = interp1d( distance, x ), interp1d( distance, y )

  count = math.ceil(pathLength/FIFTY_METERS)
  alpha = np.linspace(0, 1, count)
  xPoint, yPoint = fx(alpha), fy(alpha)

  return xPoint, yPoint

# Extracting Images

In [ ]:
processed_lat_long = []

if os.path.exists("lat_long.txt"):
    with open("lat_long.txt", "r") as f:
        processed_lat_long = [line.strip() for line in f]

print(f"Loaded {len(processed_lat_long)} latlong pairs from lat_long.txt.")

for row in less_makati_streets[250:850]['geometry'].items():
    print(f"Searching road {row[0]}.")
    x = row[1].coords.xy[1]
    y = row[1].coords.xy[0]
    length = row[1].length
    equiXPoints, equiYPoints = getEquidistantPoints(length, x, y)
    # TOO SCARED TO RUN THIS LOL
    for i in range(0, len(equiXPoints)):
      if "('" + str(equiXPoints[i]) +"', '"+ str(equiYPoints[i]) + "')" not in processed_lat_long:
        latlong = str(equiXPoints[i]) + "," + str(equiYPoints[i]) 
        print(latlong + " unprocessed.")
        status = getGSVMetadata(equiXPoints[i], equiYPoints[i])
        if status == 'DUPLICATE':
          print(str(equiXPoints[i]) + "," + str(equiYPoints[i]) + "'s pano_id has already been processed.")
          processed_lat_long.append(f"('{equiXPoints[i]}', '{equiYPoints[i]}')")
          with open("lat_long.txt", "a") as f:      # append latlong to .txt file
            f.write(f"('{equiXPoints[i]}', '{equiYPoints[i]}')\n")
        elif "NO IMAGE" in status:
          print(f"{equiXPoints[i]},{equiYPoints[i]} No image found.")
          processed_lat_long.append(f"('{equiXPoints[i]}', '{equiYPoints[i]}')")
          with open("lat_long.txt", "a") as f:      # append latlong to .txt file
            f.write(f"('{equiXPoints[i]}', '{equiYPoints[i]}')\n")
        else :
          latlong = str(equiXPoints[i]) + "," + str(equiYPoints[i]) 
          getStreet(add=latlong,outputLocation=myloc, angles=angle)
          print(latlong + " image found.")
          processed_pano_ids.append(status)         # append pano_id to processed pano list
          with open("pano_ids.txt", "a") as f:      # append pano_id to .txt file
            f.write(f"{status}\n")
          processed_lat_long.append(f"('{equiXPoints[i]}', '{equiYPoints[i]}')")
          with open("lat_long.txt", "a") as f:      # append latlong to .txt file
            f.write(f"('{equiXPoints[i]}', '{equiYPoints[i]}')\n")
            
      else:
         latlong = str(equiXPoints[i]) + "," + str(equiYPoints[i]) 
        #  print(f"{latlong} has already been processed." )

print("end.")